In [14]:
import pandas as pd 
import numpy as np 
import os
from datetime import datetime

In [10]:
os.chdir('./')

In [24]:
cab_data=pd.read_csv('dataset/cab_rides.csv')
weather_data=pd.read_csv('dataset/weather.csv')
cab_data.head(), weather_data.head()

(   distance cab_type     time_stamp    destination            source  price  \
 0      0.44     Lyft  1544952607890  North Station  Haymarket Square    5.0   
 1      0.44     Lyft  1543284023677  North Station  Haymarket Square   11.0   
 2      0.44     Lyft  1543366822198  North Station  Haymarket Square    7.0   
 3      0.44     Lyft  1543553582749  North Station  Haymarket Square   26.0   
 4      0.44     Lyft  1543463360223  North Station  Haymarket Square    9.0   
 
    surge_multiplier                                    id    product_id  \
 0               1.0  424553bb-7174-41ea-aeb4-fe06d4f4b9d7     lyft_line   
 1               1.0  4bd23055-6827-41c6-b23b-3c491f24e74d  lyft_premier   
 2               1.0  981a3613-77af-4620-a42a-0c0866077d1e          lyft   
 3               1.0  c2d88af2-d278-4bfd-a8d0-29ca77cc5512   lyft_luxsuv   
 4               1.0  e0126e1f-8ca9-4f2e-82b3-50505a09db9a     lyft_plus   
 
            name  
 0        Shared  
 1           Lux  
 2 

# convert to time_stamp to date time

In [27]:
cab_data['date_time']=pd.to_datetime(cab_data['time_stamp']/1000,unit='s')
weather_data['date_time']=pd.to_datetime(weather_data['time_stamp']/1000, unit='s')

# create a new column for merging the dataset

In [31]:
cab_data['merge_date']=cab_data['source'].astype(str)+'-'+cab_data['date_time'].dt.date.astype(str)+'-'+cab_data['date_time'].dt.hour.astype(str)
weather_data['merge_date']=weather_data['location'].astype(str)+weather_data['date_time'].dt.date.astype(str)+'-'+weather_data['date_time'].dt.hour.astype(str)

In [32]:
cab_data.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time,merge_date
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 09:30:07.890000105,Haymarket Square-2018-12-16-9
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 02:00:23.677000046,Haymarket Square-2018-11-27-2
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018-11-28 01:00:22.197999954,Haymarket Square-2018-11-28-1
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018-11-30 04:53:02.749000072,Haymarket Square-2018-11-30-4
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018-11-29 03:49:20.223000050,Haymarket Square-2018-11-29-3


# check for empty values

# merge the datasets 

In [ ]:
groupby_value=weather_data.groupby(['merge_data'])